In [1]:
import pandas as pd
import numpy as np
import random
import ipynb.fs.full.pmx_function as pmx
import math
import plotly.graph_objs as go

# Importing Dataset

In [2]:
cities = [
    'helsinki','espoo','tampere','vantaa','oulu','turku','kotka','lahti','kuopio','pori','kouvola',
    'joensuu','lappeenranta','mikkeli','vaasa'
]
dataframe = pd.read_csv('distance_table.csv')
dataframe

,helsinki,espoo,tampere,vantaa,oulu,turku,kotka,lahti,kuopio,pori,kouvola,joensuu,lappeenranta,mikkeli,vaasa
helsinki,0,16,160,15,539,150,114,99,336,225,124,373,203,211,370
espoo,16,0,151,24,536,134,128,103,339,211,134,381,215,217,359
tampere,160,151,0,150,400,142,204,116,254,106,171,335,240,186,210
vantaa,15,24,150,0,525,153,105,84,321,220,111,359,191,196,360
oulu,539,536,400,525,0,533,511,448,259,433,465,341,460,380,284
turku,150,134,142,153,533,0,255,194,394,118,246,463,328,302,296
kotka,114,128,204,105,511,255,0,89,273,299,46,281,95,137,404
lahti,99,103,116,84,448,194,89,0,237,215,58,281,136,116,316
kuopio,336,339,254,321,259,394,273,237,0,343,231,111,206,135,307
pori,225,211,106,220,433,118,299,215,343,0,272,434,345,291,180


In [3]:
# NODE DATA WITH 20 CITIES
#cities_20 = [
#    'helsinki','espoo','tampere','vantaa','oulu','turku','kotka','lahti','kuopio','pori','kouvola',
#    'joensuu','lappeenranta','mikkeli','vaasa','rovaniemi','kemi','tornio','savonlinna','nokia'
#]

#dataframe = pd.read_csv('dt_20.csv')
#dataframe.index = cities_20
#dataframe

In [4]:
fitness_table = dataframe.to_dict()

# Code:

In [5]:
def visualize(data):
    '''Function for visualizing data'''
    # PLACEHOLDER CONTAINER
    structure = []
    
    # LINE COLOURS
    colours = ['#e75f5b', '#52af52', '#f2ae42', '#333eee', '#a93581', 
               '#ffc0cb', '#928478']
    
    # CREATE LINE FOR EACH COLUMN IN THE DATAFRAME
    for index, row in enumerate(data):
        structure.append(go.Scatter(
            mode='markers',
            y=row,
            line=dict(width=1),
            marker=dict(color=colours[index]),
            opacity=0.8,
            yaxis='y2',
            name='GENERATION #' + str(index + 1)
        ))

    # LAYOUT PARAMS
    layout = go.Layout(
        yaxis = dict(domain = [0, 0.2],
        showticklabels=False),
        margin=dict(l=20, r=20, t=20, b=20)
    )
    
    # CREATE THE FIGURE
    fig = go.Figure(
        data=structure,
        layout=layout
    )
    
    # ADD BUTTON MAP
    fig.update_layout(
        xaxis=dict(
            rangeslider=dict(
                visible=True
            )
        )
    )
    
    # FINALLY SHOW THE GRAPH
    fig.show()

In [6]:
def shuffle(list_obj):
    '''Takes a list object as input, shuffles the list and returns 
    the results in a new list.
    
    Variables:
    list_obj: list object
    
    Returns:
    list object
    '''
    shuffled_list = list(list_obj)
    random.shuffle(shuffled_list)
    return shuffled_list

def init_combinations(node_list, amount):
    '''Takes a list object, shuffles said list object and returns
    x amount of new combinations of the original list.
    
    Variables: 
    node_list: list object
    amount: int
    
    Returns:
    list of lists
    '''
    combinations = []
    for i in range(amount):
        x = shuffle(node_list)
        combinations.append(x)

    return combinations

def calc_parent_fitness(parents, fitness_table):
    '''Function that calculates the fitness score of a set of parents, in our cities example
    this equates to finding the fitness (distance) for a->b->c->d...->n, etc'''
    score = 0
    for index, parent in enumerate(parents):
        
        # Check if last element in list
        if index+1 == len(parents):
            score += fitness_table[parent][parents[0]]
        else:
            score += fitness_table[parent][parents[index+1]]
            
    return score

def calc_fitness(list_of_parents, fitness_table):
    '''Function that calculates all fitness values for every parent in a generation'''
    scores = []
    for index, parents in enumerate(list_of_parents):
        score = calc_parent_fitness(parents, fitness_table)
        scores.append(score)
    
    return scores

def calc_weighting(fitness_scores):
    '''Function that calculates to fitness weighting based on all fitness scores'''
    weights = []
    percent_weights = []
    weight_sum = 0
    # CALCULATE FITNESS SCORE WEIGHTS
    for fitness in fitness_scores:
        score = 1/(1 + fitness)
        weights.append(score)
        weight_sum += score
    # CALCULATE PERCENTAGE WEIGHT
    for weight in weights:
        score = weight / weight_sum
        percent_weights.append(score)
    return percent_weights

def select_one(weighting, block_choice=np.inf):
    '''Function that selcets a value based on weighting'''
    choice = np.random.choice(len(weighting), p=weighting)
    
    # RUN UNTIL A NON-BLOCKED VALUE IS GENERATED
    while choice == block_choice:
        choice = np.random.choice(len(weighting), p=weighting)
        
    return choice
    
def generate_pairs(parents, weighting):
    '''Function gererates pairs based on the existing parents in a generation'''
    pairs = []
    
    # LOOP THROUGH POPULATION
    for index in range(int(len(parents) / 2)):
        first = select_one(weighting)
        second = select_one(weighting, first)
        
        pairs.append([
            parents[first],
            parents[second]
        ])
    
    return pairs

def generate_children(parent_pairs):
    '''Function that generates 2 child objects for each parent pair'''
    
    container = []
    
    crossover_points = breakpoint_generator(parent_pairs)
    #crossover_points = [4,8]
    
    for pair in parent_pairs:
        # PMX FUNCTION
        child_1, child_2 = pmx.pmx_crossover(pair[0], pair[1], crossover_points)
        
        container.append(child_1)
        container.append(child_2)
        
    return container

def next_generation(parents, dataframe):
    '''Function that does all operations needed to create the next generation of children'''
    
    # CALCULATE THEIR FITNESS
    fitness_values = calc_fitness(parents, dataframe)
    
    # CALCULATE FINESS WEIGHTING
    fitness_weighting = calc_weighting(fitness_values)
    
    # GENERATE PARENT PAIRS
    parent_pairs = generate_pairs(parents, fitness_weighting)
    
    # CREATE CHILDREN WITH PMX
    children = generate_children(parent_pairs)
    best_fitness_value = min(fitness_values)
    
    return children, best_fitness_value, [parents[np.argmin(fitness_values)], min(fitness_values)]

def breakpoint_generator(pairs):
    '''Function that creates random breakpoints for the PMX function'''

    # CHECK THE MAXIMUM PARENT SIZE
    size = len(pairs[0][0])
    
    # SPLIT INTO HALVES
    first_half = math.floor(size / 2)
    second_half = size - first_half
    
    # RANDOMLY CHOOSE AN INDEX FROM BOTH HALVES
    first = random.randint(0, first_half)
    second = random.randint(second_half, size - 1)
    
    return [first, second]

def run(dataframe, amount, iterations):
    '''Main Function, takes in amount of parents and iterates (generations) over them for x iterations
    
    dataframe: input data
    amount: number of parents
    iterations: number of generations
    '''
    
    results = []
    best_children = []
    node_list = dataframe.columns
    
    #convert DataFrame to dictionary
    f_table = dataframe.to_dict()
    
    # first step: shuffle cities into x new combinations
    parents = init_combinations(node_list, amount)
    
    for index in range(iterations):
        
        # GENERATE NEW PARENTS
        parents, best_result, best_child = next_generation(parents, dataframe)
        
        # APPEND THE BEST FITNESS VALUE & ROUTE
        results.append(best_result)
        best_children.append(best_child)
        
    return results, best_children
        
def engine(dataframe, amount, iterations, epochs):
    '''Master function.
    
    dataframe: input data
    amount: number of parents
    iterations: number of generations
    epochs: number of runs
    '''
    results = []
    parents = []
    for e in range(epochs):
        print(f'epoch: {e+1}')
        result, parent = run(dataframe, amount, iterations)
        results.append(result)
        parents.append(parent)
        
    return results, parents
    

## Implementation

In [13]:
res, par = engine(**{
    'dataframe': dataframe,
    'amount': 500, # number of parents
    'iterations': 500, # number of generations
    'epochs': 3 # number of runs
})

epoch: 1
epoch: 2
epoch: 3


In [14]:
# FETCH MIN VALUE FROM EACH RUN
for index, re in enumerate(res):
    print(f'epoch {index+1}: {min(re)}')

epoch 1: 1779
epoch 2: 1826
epoch 3: 1832


In [15]:
# VISUALIZING DATA
visualize([res[0],res[1],res[2]])